In [70]:
CUDA_VISIBLE_DEVICES=0,1
import sys
sys.path.append('/media/dataserver/workspace/blanca/utils/')

from utils import libraries
# from utils import utils_global
from utils_global import *

%matplotlib inline
import imageio

In [71]:

#########

from __future__ import print_function
import argparse
import torch
import torch.nn.parallel

import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torchvision.utils as vutils
from torch.autograd import Variable

parser = argparse.ArgumentParser()
parser.add_argument('--dataset', required=True,
                    help='cifar10 | lsun | imagenet | folder | lfw ')
parser.add_argument('--dataroot', type=str, help='path to dataset')
parser.add_argument('--workers', type=int,
                    help='number of data loading workers', default=8)
parser.add_argument('--batch_size', type=int,
                    default=64, help='batch size')
parser.add_argument('--image_size', type=int, default=32,
                    help='the resolution of the input image to network')
parser.add_argument('--nz', type=int, default=100,
                    help='size of the latent z vector')
parser.add_argument('--ngf', type=int, default=64)
parser.add_argument('--ndf', type=int, default=64)
parser.add_argument('--nc', type=int)

parser.add_argument('--nepoch', type=int, default=25,
                    help='number of epochs to train for')
parser.add_argument('--lr', type=float, default=0.0002,
                    help='learning rate, default=0.0002')
parser.add_argument('--beta1', type=float, default=0.5,
                    help='beta1 for adam. default=0.5')
parser.add_argument('--cpu', action='store_true',
                    help='use CPU instead of GPU')
parser.add_argument('--ngpu', type=int, default=1,
                    help='number of GPUs to use')

parser.add_argument('--netG', default='',
                    help="path to netG config")
parser.add_argument('--netE', default='',
                    help="path to netE config")
parser.add_argument('--netG_chp', default='',
                    help="path to netG (to continue training)")
parser.add_argument('--netE_chp', default='',
                    help="path to netE (to continue training)")

parser.add_argument('--save_dir', default='.',
                    help='folder to output images and model checkpoints')
parser.add_argument('--criterion', default='param',
                    help='param|nonparam, How to estimate KL')
parser.add_argument('--KL', default='qp', help='pq|qp')
parser.add_argument('--noise', default='sphere', help='normal|sphere')
parser.add_argument('--match_z', default='cos', help='none|L1|L2|cos')
parser.add_argument('--match_x', default='L1', help='none|L1|L2|cos')

parser.add_argument('--drop_lr', default=5, type=int, help='')
parser.add_argument('--save_every', default=50, type=int, help='')

parser.add_argument('--manual_seed', type=int, default=123, help='manual seed')
parser.add_argument('--start_epoch', type=int, default=0, help='epoch number to start with')

parser.add_argument(
    '--e_updates', default="1;KL_fake:1,KL_real:1,match_z:0,match_x:0",
    help='Update plan for encoder <number of updates>;[<term:weight>]'
)

parser.add_argument(
    '--g_updates', default="2;KL_fake:1,match_z:1,match_x:0",
    help='Update plan for generator <number of updates>;[<term:weight>]'
)

torch.cuda.is_available()

True

#### Setting up the options

In [72]:
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.utils.data
import importlib
# from .dataset import FolderWithImages
import random
import torch.backends.cudnn as cudnn
from PIL import Image

In [73]:
# setup function @utils.py

cuda = torch.cuda.is_available() # not opt.cpu
torch.set_num_threads(4)
dataset = 'latest_v2.4'

save_dir = 'output/' + dataset
try:
    os.makedirs(save_dir)
except OSError:
    print('Directory was not created.')

manual_seed = random.randint(1, 10000)
random.seed(manual_seed)
torch.manual_seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

cudnn.benchmark = True

if torch.cuda.is_available() and not cuda:
    print("WARNING: You have a CUDA device,"
            "so you should probably run with --cuda")

e_updates = '1;KL_fake:1,KL_real:1,match_z:0,match_x:10' 
g_updates = '3;KL_fake:1,match_z:1000,match_x:0'
    
updates = {'e': {}, 'g': {}}
updates['e']['num_updates'] = int(e_updates.split(';')[0])
updates['e'].update({x.split(':')[0]: float(x.split(':')[1]) 
                     for x in e_updates.split(';')[1].split(',')})

updates['g']['num_updates'] = int(g_updates.split(';')[0])
updates['g'].update({x.split(':')[0]: float(x.split(':')[1]) 
                     for x in g_updates.split(';')[1].split(',')})


###################

image_size = 64 # 512
batch_size = 128 # 64
workers = 16
shuffle = True
drop_last = True
train = True

nc = 3
nz = 64 #100 #'size of the latent z vector')
ngf = 64
ndf = 64
ngpu = 2 # 'number of GPUs to use')
pin_memory = True # True | If ``True``, the data loader will copy tensors into CUDA pinned memory before returning them.

noise = 'sphere' #help='normal|sphere')

# netG = 'dcgan128px' 
# netE = 'dcgan128px' 

netG_chp = '' # "path to netG (to continue training)"
netE_chp = '' # "path to netE (to continue training)"

lr = 0.0002
drop_lr = 100
beta1 = 0.5 # help='beta1 for adam. default=0.5'
criterion = 'param'# help='param|nonparam, How to estimate KL'
KL ='qp' # help='pq|qp'
match_z = 'cos' # help='none|L1|L2|cos'
match_x = 'L1' # help='none|L1|L2|cos'

start_epoch = 0
nepoch = 1000

save_every_e = 1
save_every_b = None
dataroot = '/media/dataserver/workspace/blanca/checkpoints/AGE-128/AGE/data/training/'

run = '%d_%d_%d_nz%d_lrdlr_%d_%d' %(image_size, batch_size, nepoch, nz, lr, drop_lr); print(run)

# --lr 0.0002 
# --nz 64 
# --batch_size 64 
# --netG dcgan64px 
# --netE dcgan64px 
# --nepoch 5 
# --drop_lr 5 
# --e_updates '1;KL_fake:1,KL_real:1,match_z:0,match_x:10' 
# --g_updates '3;KL_fake:1,match_z:1000,match_x:0'

Directory was not created.
64_128_1000_nz64_lrdlr_0_100


In [74]:
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.utils.data
import importlib
import random
import os
import torch.backends.cudnn as cudnn
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

##
import torch.utils.data as data
from os import listdir
from os.path import join
from PIL import Image

def is_image_file(filename):
    return any(filename.endswith(extension) for extension in [".png", ".jpg", ".jpeg"])

def load_img(filepath):
#     img = cv2.imread(filepath) #, cv2.IMREAD_UNCHANGED)
    img = Image.open(filepath).convert('RGB')
    return img

class FolderWithImages(data.Dataset):
    def __init__(self, root, input_transform=None, target_transform=None):
        super(FolderWithImages, self).__init__()
        self.image_filenames = [join(root, x)
                                for x in listdir(root) if is_image_file(x.lower())]

        self.input_transform = input_transform
        self.target_transform = target_transform

    def __getitem__(self, index):
        input = load_img(self.image_filenames[index])
        target = input.copy()
        if self.input_transform:
            input = self.input_transform(input)
        if self.target_transform:
            target = self.target_transform(target)

        return input, target

    def __len__(self):
        return len(self.image_filenames)

    
def setup_dataset(dataset, dataroot, train=True, shuffle=True, drop_last=True):
    '''
    Setups dataset.
    '''
    # Usual transform
    t = transforms.Compose([
        transforms.Scale([image_size, image_size]),
        transforms.ToTensor(),
        transforms.Normalize(
            (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

    if dataset in ['imagenet', 'folder', 'lfw']:
        imdir = 'train' if train else 'val'
        dataroot = os.path.join(dataroot, imdir)

        dataset = dset.ImageFolder(root=dataroot, transform=t)
    elif dataset == 'lsun':
        dataset = dset.LSUN(db_path=dataroot,
                            classes=['bedroom_train'],
                            train=train,
                            transform=t)
    elif dataset == 'cifar10':
        dataset = dset.CIFAR10(root='data/raw/cifar10',
                               download=True,
                               train=train,
                               transform=t
                               )
    elif dataset == 'mnist':
        dataset = dset.MNIST(root='data/raw/mnist',
                             download=True,
                             train=train,
                             transform=t
                             )
    elif dataset == 'svhn':
        dataset = dset.SVHN(root='data/raw/svhn',
                            download=True,
                            train=train,
                            transform=t)
    elif dataset == 'celeba':
        imdir = 'train' if train else 'val'
        dataroot = os.path.join(dataroot, imdir)

        dataset = FolderWithImages(root=dataroot,
                                   input_transform=transforms.Compose([
                                       ALICropAndScale(),
                                       transforms.ToTensor(),
                                       transforms.Normalize(
                                           (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                                   ]),
                                   target_transform=transforms.ToTensor()
                                   )
    elif dataset == 'helen':
        imdir = 'train' if train else 'val'
        dataroot = os.path.join(dataroot, imdir)

        dataset = FolderWithImages(root=dataroot,
                                   input_transform=transforms.Compose([
                                       Scale(),
                                       transforms.ToTensor(),
                                       transforms.Normalize(
                                           (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                                   ]),
                                   target_transform=transforms.ToTensor()
                                   )
        
    else:
        imdir = 'train' if train else 'val'
        dataroot = os.path.join(dataroot, imdir)

        dataset = FolderWithImages(root=dataroot,
                                   input_transform=transforms.Compose([
                                       Scale(),
                                       transforms.ToTensor(),
                                       transforms.Normalize(
                                           (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                                   ]),
                                   target_transform=transforms.ToTensor()
                                   )
        
#     else:
#         assert False, 'Wrong dataset name.'

    assert len(dataset) > 0, 'No images found, check your paths.'

    # Shuffle and drop last when training
    dataloader = torch.utils.data.DataLoader(dataset,
                                             batch_size=batch_size,
                                             shuffle=shuffle,
                                             num_workers=int(workers),
                                             pin_memory=pin_memory,
                                             drop_last=drop_last)
#     print(len(dataloader))
#     for i in dataloader: print(i[0].shape, i[1].shape)
        
    return InfiniteDataLoader(dataloader)
#     return dataloader

class InfiniteDataLoader(object):
    """docstring for InfiniteDataLoader"""

    def __init__(self, dataloader):
        super(InfiniteDataLoader, self).__init__()
        self.dataloader = dataloader
        self.data_iter = None

    def next(self):
        try:
            data = self.data_iter.next()
        except Exception:
            # Reached end of the dataset
            self.data_iter = iter(self.dataloader)
            data = self.data_iter.next()

        return data

    def __len__(self):
        return len(self.dataloader)

class ALICropAndScale(object):
    def __call__(self, img):
        return img.resize((64, 78), Image.ANTIALIAS).crop((0, 7, 64, 64 + 7))
    
class Scale(object):
    def __call__(self, img):
#         img = cv2.resize(img, (64, 64), cv2.INTER_AREA)
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#         img = Image.fromarray(img)
        img = img.resize((image_size, image_size), Image.ANTIALIAS)
        return img 

# cv::INTER_AREA interpolation, whereas to
# .   enlarge an image, it will generally look best with cv::INTER_CUBIC (slow) or cv::INTER_LINEAR
# .   (faster but still looks OK).
    
# Setup dataset
dataloader = dict(train=setup_dataset(dataset, dataroot, train=True),
                  val=setup_dataset(dataset, dataroot, train=False))

/root/miniconda3/lib/python3.6/site-packages/torchvision-0.2.0-py3.6.egg/torchvision/transforms/transforms.py:156: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.


In [75]:
def normalize(x, dim=0):
    '''
    Projects points to a sphere.
    '''
    return x.div(x.norm(2, dim=dim).expand_as(x))

def normalize_(x, dim=0):
    '''
    Projects points to a sphere inplace.
    '''
    x.div_(x.norm(2, dim=dim).expand_as(x))

In [76]:
def weights_init(m):
    '''
    Custom weights initialization called on netG and netE
    '''
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

class _netE_Base(nn.Module):
    def __init__(self, main):
        super(_netE_Base, self).__init__()
        self.noise = noise
        self.ngpu = ngpu
        self.main = main

    def forward(self, input):
        gpu_ids = None
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 0:
            gpu_ids = range(self.ngpu)
            output = nn.parallel.data_parallel(self.main, input, gpu_ids)
        else:
            output = self.main(input)
        output = output.view(output.size(0), -1)
        if self.noise == 'sphere':
            output = normalize(output)
        return output
    
class _netG_Base(nn.Module):
    def __init__(self, main):
        super(_netG_Base, self).__init__()
        self.ngpu = ngpu
        self.main = main

    def forward(self, input):

        # Check input is either (B,C,1,1) or (B,C)
        assert input.nelement() == input.size(0) * input.size(1), 'wtf'
        input = input.view(input.size(0), input.size(1), 1, 1)

        gpu_ids = None
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 0:
            gpu_ids = range(self.ngpu)
            return nn.parallel.data_parallel(self.main, input, gpu_ids)
        else:
            return self.main(input)

if image_size == 128:
    def _netE():
        main = nn.Sequential(
            # input is (nc) x 128 x 128
            # state size. (ndf) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 4 x 4
            nn.Conv2d(ndf * 8, ndf * 16, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 16),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 1 x 1
            nn.Conv2d(ndf * 16, nz, 4, 1, 0, bias=True),
        )

        return _netE_Base(main)

    def _netG():
        main = nn.Sequential(
            # input is Z, going into a convolution
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(nz, ngf * 16, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 16),
            nn.ReLU(True),
            # state size. (ngf*8) x 8 x 8
            nn.ConvTranspose2d(ngf * 16, ngf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 16 x 16
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 32 x 32
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 64 x 64
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 128 x 128
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )

        return _netG_Base(main)

elif image_size == 64:
    def _netE():
        main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 16 x 16
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 8 x 8
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 4 x 4
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, nz, 4, 1, 0, bias=True),
        )
        return _netE_Base(main)

    def _netG():
        main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

        return _netG_Base(main)
    
    
def load_G():
    '''
    Loads generator model.
    '''
    netG = _netG()
    netG.apply(weights_init)
    netG.train()
    if netG_chp != '':
        netG.load_state_dict(torch.load(netG_chp).state_dict())

    print('Generator\n', netG)
    return netG

def load_E():
    '''
    Loads encoder model.
    '''
    netE = _netE()
    netE.apply(weights_init)
    netE.train()
    if netE_chp != '':
        netE.load_state_dict(torch.load(netE_chp).state_dict())

    print('Encoder\n', netE)

    return netE

# Load generator
netG = load_G()

# Load encoder
netE = load_E()

# RuntimeError: Given transposed=1, weight[64, 512, 4, 4], so expected input[64, 256, 1, 1] to have 64 channels,
# but got 256 channels instead

Generator
 _netG_Base(
  (main): Sequential(
    (0): ConvTranspose2d(64, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU(inplace)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (5): ReLU(inplace)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (8): ReLU(inplace)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (11): ReLU(inplace)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)
Encoder
 _netE_Base(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 

#### Pretraining preparation

In [77]:
def var(x, dim=0):
    '''
    Calculates variance.
    '''
    x_zero_meaned = x - x.mean(dim).expand_as(x)
    return x_zero_meaned.pow(2).mean(dim)


class KLN01Loss(torch.nn.Module):

    def __init__(self, direction, minimize):
        super(KLN01Loss, self).__init__()
        self.minimize = minimize
        assert direction in ['pq', 'qp'], 'direction?'

        self.direction = direction

    def forward(self, samples):

        assert samples.nelement() == samples.size(1) * samples.size(0), 'wtf?'

        samples = samples.view(samples.size(0), -1)

        self.samples_var = var(samples)
        self.samples_mean = samples.mean(0)

        samples_mean = self.samples_mean
        samples_var = self.samples_var

        if self.direction == 'pq':
            # mu_1 = 0; sigma_1 = 1

            t1 = (1 + samples_mean.pow(2)) / (2 * samples_var.pow(2))
            t2 = samples_var.log()

            KL = (t1 + t2 - 0.5).mean()
        else:
            # mu_2 = 0; sigma_2 = 1

            t1 = (samples_var.pow(2) + samples_mean.pow(2)) / 2
            t2 = -samples_var.log()

            KL = (t1 + t2 - 0.5).mean()

        if not self.minimize:
            KL *= -1

        return KL

# ----------------------------

def pairwise_euclidean(samples):

    B = samples.size(0)

    samples_norm = samples.mul(samples).sum(1)
    samples_norm = samples_norm.expand(B, B)

    dist_mat = samples.mm(samples.t()).mul(-2) + \
        samples_norm.add(samples_norm.t())
    return dist_mat

def sample_entropy(samples):

        # Assume B x C input

    dist_mat = pairwise_euclidean(samples)

    # Get max and add it to diag
    m = dist_mat.max().detach()
    dist_mat_d = dist_mat + \
        Variable(torch.eye(dist_mat.size(0)) * (m.data[0] + 1)).cuda()

    entropy = (dist_mat_d.min(1)[0] + 1e-4).log().sum()

    entropy *= (samples.size(1) + 0.) / samples.size(0)

    return entropy

class SampleKLN01Loss(torch.nn.Module):

    def __init__(self, direction, minimize):
        super(SampleKLN01Loss, self).__init__()
        self.minimize = minimize
        assert direction in ['pq', 'qp'], 'direction?'

        self.direction = direction

    def forward(self, samples):

        assert samples.ndimension == 2, 'wft'
        samples = samples.view(samples.size(0), -1)

        self.samples_var = var(samples)
        self.samples_mean = samples.mean(0)

        if self.direction == 'pq':
            assert False, 'not possible'
        else:
            entropy = sample_entropy(samples)

            cross_entropy = - samples.pow(2).mean() / 2.

            KL = - cross_entropy - entropy

        if not self.minimize:
            KL *= -1

        return KL

In [78]:
# x = torch.FloatTensor(batch_size, nc, image_size, image_size)
# z = torch.FloatTensor(batch_size, nz, 1, 1)
# fixed_z = torch.FloatTensor(batch_size, nz, 1, 1).normal_(0, 1)
# z = fixed_z

In [79]:
x = torch.FloatTensor(batch_size, nc, image_size, image_size)
z = torch.FloatTensor(batch_size, nz, 1, 1)
fixed_z = torch.FloatTensor(batch_size, nz, 1, 1).normal_(0, 1)

def match(x, y, dist):
    '''
    Computes distance between corresponding points in `x` and `y`
    using distance `dist`.
    '''
    if dist == 'L2':
        return (x - y).pow(2).mean()
    elif dist == 'L1':
        return (x - y).abs().mean()
    elif dist == 'cos':
        x_n = normalize(x)
        y_n = normalize(y)

        return 2 - (x_n).mul(y_n).mean()
    else:
        assert dist == 'none', 'wtf ?'

def normalize(x, dim=0):
    '''
    Projects points to a sphere.
    '''
    return x.div(x.norm(2, dim=dim).expand_as(x))

def normalize_(x, dim=0):
    '''
    Projects points to a sphere inplace.
    '''
    x.div_(x.norm(2, dim=dim).expand_as(x))

if noise == 'sphere':
    normalize_(fixed_z)

if cuda:
    netE.cuda()
    netG.cuda()
    x = x.cuda()
    z, fixed_z = z.cuda(), fixed_z.cuda()

x = Variable(x)
z = Variable(z)
fixed_z = Variable(fixed_z)

# Setup optimizers
optimizerD = optim.Adam(netE.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

# Setup criterions
if criterion == 'param':
    print('Using parametric criterion KL_%s' % KL)
    KL_minimizer = KLN01Loss(direction=KL, minimize=True)
    KL_maximizer = KLN01Loss(direction=KL, minimize=False)
elif criterion == 'nonparam':
    print('Using NON-parametric criterion KL_%s' % KL)
    KL_minimizer = SampleKLN01Loss(direction=KL, minimize=True)
    KL_maximizer = SampleKLN01Loss(direction=KL, minimize=False)
else:
    assert False, 'criterion?'

real_cpu = torch.FloatTensor()


def save_image(tensor, filename, nrow=8, padding=2,
               normalize=False, range=None, scale_each=False, pad_value=0):
    """Save a given Tensor into an image file.
    Args:
        tensor (Tensor or list): Image to be saved. If given a mini-batch tensor,
            saves the tensor as a grid of images by calling ``make_grid``.
        **kwargs: Other arguments are documented in ``make_grid``.
    """
    from PIL import Image
    grid = vutils.make_grid(tensor, nrow=nrow, padding=padding, pad_value=pad_value,
                     normalize=normalize, range=range, scale_each=scale_each)
    ndarr = grid.mul(255).clamp(0, 255).byte().permute(1, 2, 0).cpu().numpy() # 
    im = Image.fromarray(ndarr)
#     im = Image.fromarray(ndarr[:,:,::-1])    
    im.save(filename)


def save_images(epoch):

    real_cpu.resize_(x.data.size()).copy_(x.data)

    # Real samples
    save_path = '%s/real_samples.png' % save_dir
    save_image(real_cpu[:64] / 2 + 0.5, save_path)

    netG.eval()
    fake = netG(fixed_z)

    # Fake samples
    save_path = '%s/fake_samples_epoch_%03d.png' % (save_dir, epoch)
    save_image(fake.data[:64] / 2 + 0.5, save_path)

    # Save reconstructions
    populate_x(x, dataloader['val'])
    gex = netG(netE(x)) # here - at G entry

    t = torch.FloatTensor(x.size(0) * 2, x.size(1),
                          x.size(2), x.size(3))

    t[0::2] = x.data[:]
    t[1::2] = gex.data[:]

    save_path = '%s/reconstructions_epoch_%03d.png' % (save_dir, epoch)
    grid = save_image(t[:64] / 2 + 0.5, save_path)
    
    netG.train()

def adjust_lr(epoch, drop_coef=2):
    if (epoch + 1) % (drop_lr + 1) == 0:
        ###
        assert optimizerD.param_groups[0]['lr'] == optimizerG.param_groups[0]['lr']
        lr = optimizerD.param_groups[0]['lr']
        print('Adjusting learning rate from %f to %f on E and G' % (lr, lr / drop_coef))
        optimizerD.param_groups[0]['lr'] /= drop_coef
        optimizerG.param_groups[0]['lr'] /= drop_coef
        ###
#         lr /= 2
#         for param_group in optimizerD.param_groups:
#             param_group['lr'] = lr

#         for param_group in optimizerG.param_groups:
#             param_group['lr'] = lr



Using parametric criterion KL_qp


In [80]:
def populate_x(x, dataloader):
    '''
    Fills input variable `x` with data generated with dataloader
    '''
    real_cpu, _ = dataloader.next()
    x.data.resize_(real_cpu.size()).copy_(real_cpu)

def populate_z(z):
    '''
    Fills noise variable `z` with noise U(S^M)
    '''
    z.data.resize_(batch_size, nz, 1, 1)
    z.data.normal_(0, 1)
    if noise == 'sphere':
        normalize_(z.data)

In [81]:
# print(len(dataloader['train'].next()))
# print(len(dataloader['val'].next()))
# print(dataloader['train'].next()[0].shape, dataloader['train'].next()[1].shape)
# print(dataloader['train'].next()[0].shape, dataloader['train'].next()[1].shape)

# populate_x(x, dataloader['val'])
# nepoch = 10
# lr = 0.0002

In [ ]:
stats = {}
losses = {'KL_real': [],
         'Ex_e': [],
         'KL_fake': [], 
         'Ez_e': [],
         'KL_fake_g': [],
         'Ex_g: ': [],
         'Ez_g_10e3': [],
        }

for epoch in range(start_epoch, nepoch):

    # Adjust learning rate
    adjust_lr(epoch)

    for i in range(len(dataloader['train'])):

        # ---------------------------
        #        Optimize over e
        # ---------------------------
        
        # e_updates = '1; KL_fake:1, KL_real:1, match_z:0, match_x:10' 
        
        KL_real_ = []
        KL_fake_ = []
        Ex_e_ = []
        Ez_e_ = []

        for e_iter in range(updates['e']['num_updates']):
            e_losses = []
            netE.zero_grad()

            # X - e(X)
            populate_x(x, dataloader['train'])
            ex = netE(x)

            # KL_real: - \Delta( e(X) , Z ) -> max_e
            KL_real = KL_minimizer(ex)
            e_losses.append(KL_real * updates['e']['KL_real'])
            # === stats ===    
            KL_real_.append(KL_real.data[0])
            
            if updates['e']['match_x'] != 0:
                # g(e(X))
                gex = netG(ex)

                # match_x: E_x||g(e(x)) - x|| -> min_e
                err = match(gex, x, match_x)
                e_losses.append(err * 10) #updates['e']['match_x'])
                # === stats === 
                Ex_e_.append(err.data[0])

            # Save some stats
            stats['real_mean'] = KL_minimizer.samples_mean.data.mean()
            stats['real_var'] = KL_minimizer.samples_var.data.mean()
            stats['KL_real'] = KL_real.data[0]

            # ================================================

            # Z - g(Z) - e(g(Z))
            populate_z(z)
            fake = netG(z).detach()
            egz = netE(fake)

            # KL_fake: \Delta( e(g(Z)) , Z ) -> max_e
            KL_fake = KL_maximizer(egz)
            e_losses.append(KL_fake * updates['e']['KL_fake'])
            # === stats === 
            KL_fake_.append(-KL_fake.data[0])
            
            if updates['e']['match_z'] != 0:
                # match_z: E_z||e(g(z)) - z|| -> min_e
                err = match(egz, z, match_z)
                e_losses.append(err * updates['e']['match_z'])
                # === stats === 
                Ez_e_.append(err.data[0])
            
            # Update e
            sum(e_losses).backward()
            optimizerD.step()

            # === stats === 
            stats['fake_mean'] = KL_maximizer.samples_mean.data.mean()
            stats['fake_var'] = KL_maximizer.samples_var.data.mean()
            stats['KL_fake'] = -KL_fake.data[0]
            
            if KL_real_: losses['KL_real'].append(np.mean(KL_real_))
            if KL_fake_: losses['KL_fake'].append(np.mean(KL_fake_))
            if Ex_e_: losses['Ex_e'].append(np.mean(Ex_e_))
            if Ez_e_: losses['Ez_e'].append(np.mean(Ez_e_))
            
        # ---------------------------
        #        Minimize over g
        # ---------------------------
        
        # g_updates = '3; KL_fake:1, match_z:1000, match_x:0'

        KL_fake_g_ = []
        Ex_g_ = []
        Ez_g_ = []
        
        for g_iter in range(updates['g']['num_updates']):
            g_losses = []
            netG.zero_grad()

            # Z - g(Z) - e(g(Z))
            populate_z(z)
            fake = netG(z)
            egz = netE(fake)

            # KL_fake: \Delta( e(g(Z)) , Z ) -> min_g
            KL_fake_g = KL_minimizer(egz)
            g_losses.append(KL_fake_g * updates['g']['KL_fake'])
            # === stats === 
            KL_fake_g_.append(KL_fake_g.data[0])
            
            if updates['g']['match_z'] != 0:
                # match_z: E_z||e(g(z)) - z|| -> min_g
                err = match(egz, z, match_z)
                err = err * updates['g']['match_z']
                g_losses.append(err)
                # === stats === 
#                 Ez_g_.append(err.data[0])
                Ez_g_.append(err.data[0] / 1000)
                
            # ==================================

            if updates['g']['match_x'] != 0:
                # X - e(X) - g(e(X))
                populate_x(x, dataloader['train'])
                ex = netE(x)
                gex = netG(ex)

                # match_x: E_x||g(e(x)) - x|| -> min_g
                err = match(gex, x, match_x)
                err = err * updates['g']['match_x']
                g_losses.append(err)
                # === stats === 
                Ex_g_.append(err.data[0])

            # Step g
            sum(g_losses).backward()
            optimizerG.step()
        
        # === stats === 
        if KL_fake_g_: losses['KL_fake_g'].append(np.mean(KL_fake_g_))
        if Ez_g_: losses['Ez_g_10e3'].append(np.mean(Ez_g_))
        if Ex_g_: losses['Ex_g'].append(np.mean(Ex_g_))
        
        # === saving === 
        
#         if save_every_b:
#             if i % save_every_b == 0: save_images(epoch)
            
        # If an epoch takes long time, dump intermediate
        if dataset in ['lsun', 'imagenet'] and (i % 5000 == 0):
            torch.save(netG, '%s/netG_epoch_%d_it_%d.pth' %
                       (save_dir, epoch, i))
            torch.save(netE, '%s/netE_epoch_%d_it_%d.pth' %
                       (save_dir, epoch, i))
    
    if epoch % save_every_e == 0: 
        # echo:
        print('[{epoch}/{nepoch}][{iter}/{niter}] '
              'KL_real/fake: {KL_real:.3f}/{KL_fake:.3f} '
              'mean_real/fake: {real_mean:.3f}/{fake_mean:.3f} '
              'var_real/fake: {real_var:.3f}/{fake_var:.3f} '
              ''.format(epoch=epoch,
                        nepoch=nepoch,
                        iter=i,
                        niter=len(dataloader['train']),
                        **stats))
        save_images(epoch)
        # checkpoint:
#         torch.save(netG, '%s/netG_epoch_%d.pth' % (save_dir, epoch))
#         torch.save(netE, '%s/netE_epoch_%d.pth' % (save_dir, epoch))

[0/1000][15/16] KL_real/fake: 4.378/4.386 mean_real/fake: -0.002/-0.005 var_real/fake: 0.008/0.008 
[1/1000][15/16] KL_real/fake: 4.464/4.938 mean_real/fake: -0.008/-0.011 var_real/fake: 0.007/0.005 
[2/1000][15/16] KL_real/fake: 4.837/5.249 mean_real/fake: -0.008/-0.007 var_real/fake: 0.005/0.004 
[3/1000][15/16] KL_real/fake: 5.010/4.934 mean_real/fake: -0.014/-0.012 var_real/fake: 0.005/0.005 
[4/1000][15/16] KL_real/fake: 4.934/4.912 mean_real/fake: -0.015/-0.012 var_real/fake: 0.005/0.005 
[5/1000][15/16] KL_real/fake: 4.984/4.958 mean_real/fake: -0.014/-0.013 var_real/fake: 0.005/0.005 
[6/1000][15/16] KL_real/fake: 4.930/4.882 mean_real/fake: -0.012/-0.011 var_real/fake: 0.005/0.005 
[7/1000][15/16] KL_real/fake: 4.948/4.908 mean_real/fake: -0.012/-0.010 var_real/fake: 0.005/0.005 
[8/1000][15/16] KL_real/fake: 4.896/4.846 mean_real/fake: -0.010/-0.009 var_real/fake: 0.006/0.006 
[9/1000][15/16] KL_real/fake: 4.916/4.883 mean_real/fake: -0.012/-0.011 var_real/fake: 0.005/0.006 


[82/1000][15/16] KL_real/fake: 4.581/4.572 mean_real/fake: 0.001/-0.001 var_real/fake: 0.006/0.006 
[83/1000][15/16] KL_real/fake: 4.576/4.564 mean_real/fake: -0.001/-0.002 var_real/fake: 0.007/0.007 
[84/1000][15/16] KL_real/fake: 4.572/4.561 mean_real/fake: -0.005/-0.002 var_real/fake: 0.006/0.007 
[85/1000][15/16] KL_real/fake: 4.588/4.573 mean_real/fake: 0.002/0.001 var_real/fake: 0.006/0.007 
[86/1000][15/16] KL_real/fake: 4.594/4.577 mean_real/fake: -0.000/0.000 var_real/fake: 0.006/0.006 
[87/1000][15/16] KL_real/fake: 4.591/4.577 mean_real/fake: -0.002/-0.002 var_real/fake: 0.006/0.006 
[88/1000][15/16] KL_real/fake: 4.597/4.581 mean_real/fake: -0.002/-0.001 var_real/fake: 0.006/0.006 
[89/1000][15/16] KL_real/fake: 4.571/4.561 mean_real/fake: -0.001/0.001 var_real/fake: 0.007/0.007 
[90/1000][15/16] KL_real/fake: 4.588/4.573 mean_real/fake: 0.002/0.001 var_real/fake: 0.006/0.006 
[91/1000][15/16] KL_real/fake: 4.585/4.570 mean_real/fake: -0.002/-0.003 var_real/fake: 0.006/0.00

[163/1000][15/16] KL_real/fake: 4.463/4.453 mean_real/fake: -0.001/0.001 var_real/fake: 0.007/0.007 
[164/1000][15/16] KL_real/fake: 4.455/4.444 mean_real/fake: 0.002/0.001 var_real/fake: 0.007/0.007 
[165/1000][15/16] KL_real/fake: 4.444/4.441 mean_real/fake: -0.000/0.001 var_real/fake: 0.007/0.007 
[166/1000][15/16] KL_real/fake: 4.452/4.441 mean_real/fake: -0.000/0.002 var_real/fake: 0.007/0.007 
[167/1000][15/16] KL_real/fake: 4.445/4.439 mean_real/fake: 0.002/0.001 var_real/fake: 0.007/0.007 
[168/1000][15/16] KL_real/fake: 4.446/4.440 mean_real/fake: -0.001/0.001 var_real/fake: 0.007/0.007 
[169/1000][15/16] KL_real/fake: 4.454/4.441 mean_real/fake: 0.000/-0.000 var_real/fake: 0.007/0.007 
[170/1000][15/16] KL_real/fake: 4.450/4.438 mean_real/fake: 0.002/0.001 var_real/fake: 0.007/0.007 
[171/1000][15/16] KL_real/fake: 4.447/4.441 mean_real/fake: 0.001/0.001 var_real/fake: 0.007/0.007 
[172/1000][15/16] KL_real/fake: 4.444/4.437 mean_real/fake: -0.001/0.000 var_real/fake: 0.007/0

[244/1000][15/16] KL_real/fake: 4.380/4.377 mean_real/fake: 0.002/0.002 var_real/fake: 0.008/0.008 
[245/1000][15/16] KL_real/fake: 4.376/4.371 mean_real/fake: 0.002/0.002 var_real/fake: 0.008/0.008 
[246/1000][15/16] KL_real/fake: 4.382/4.375 mean_real/fake: 0.001/0.002 var_real/fake: 0.008/0.008 
[247/1000][15/16] KL_real/fake: 4.378/4.376 mean_real/fake: 0.001/-0.000 var_real/fake: 0.008/0.008 
[248/1000][15/16] KL_real/fake: 4.381/4.376 mean_real/fake: -0.000/0.001 var_real/fake: 0.008/0.008 
[249/1000][15/16] KL_real/fake: 4.382/4.375 mean_real/fake: 0.002/0.001 var_real/fake: 0.008/0.008 
[250/1000][15/16] KL_real/fake: 4.379/4.372 mean_real/fake: -0.001/-0.000 var_real/fake: 0.008/0.008 
[251/1000][15/16] KL_real/fake: 4.381/4.374 mean_real/fake: 0.001/0.001 var_real/fake: 0.008/0.008 
[252/1000][15/16] KL_real/fake: 4.374/4.373 mean_real/fake: 0.000/-0.000 var_real/fake: 0.008/0.008 
[253/1000][15/16] KL_real/fake: 4.379/4.374 mean_real/fake: 0.002/0.002 var_real/fake: 0.008/0.

[325/1000][15/16] KL_real/fake: 4.370/4.366 mean_real/fake: 0.000/-0.000 var_real/fake: 0.008/0.008 
[326/1000][15/16] KL_real/fake: 4.367/4.366 mean_real/fake: 0.000/0.000 var_real/fake: 0.008/0.008 
[327/1000][15/16] KL_real/fake: 4.366/4.363 mean_real/fake: 0.001/-0.001 var_real/fake: 0.008/0.008 
[328/1000][15/16] KL_real/fake: 4.369/4.366 mean_real/fake: 0.001/0.002 var_real/fake: 0.008/0.008 
[329/1000][15/16] KL_real/fake: 4.370/4.363 mean_real/fake: -0.001/0.002 var_real/fake: 0.008/0.008 
[330/1000][15/16] KL_real/fake: 4.367/4.361 mean_real/fake: -0.001/0.000 var_real/fake: 0.008/0.008 
[331/1000][15/16] KL_real/fake: 4.365/4.363 mean_real/fake: 0.000/-0.000 var_real/fake: 0.008/0.008 
[332/1000][15/16] KL_real/fake: 4.363/4.362 mean_real/fake: 0.000/-0.000 var_real/fake: 0.008/0.008 
[333/1000][15/16] KL_real/fake: 4.370/4.363 mean_real/fake: 0.001/0.001 var_real/fake: 0.008/0.008 
[334/1000][15/16] KL_real/fake: 4.368/4.363 mean_real/fake: 0.000/0.002 var_real/fake: 0.008/0

...

[569/1000][15/16] KL_real/fake: 4.362/4.358 mean_real/fake: -0.000/0.000 var_real/fake: 0.008/0.008 
[570/1000][15/16] KL_real/fake: 4.360/4.359 mean_real/fake: -0.000/0.001 var_real/fake: 0.008/0.008 
[571/1000][15/16] KL_real/fake: 4.361/4.359 mean_real/fake: -0.001/0.000 var_real/fake: 0.008/0.008 
[572/1000][15/16] KL_real/fake: 4.361/4.359 mean_real/fake: 0.000/0.001 var_real/fake: 0.008/0.008 
[573/1000][15/16] KL_real/fake: 4.364/4.358 mean_real/fake: 0.000/-0.001 var_real/fake: 0.008/0.008 
[574/1000][15/16] KL_real/fake: 4.362/4.359 mean_real/fake: -0.001/0.000 var_real/fake: 0.008/0.008 
[575/1000][15/16] KL_real/fake: 4.365/4.358 mean_real/fake: 0.000/-0.000 var_real/fake: 0.008/0.008 
[576/1000][15/16] KL_real/fake: 4.362/4.359 mean_real/fake: 0.000/0.001 var_real/fake: 0.008/0.008 
[577/1000][15/16] KL_real/fake: 4.362/4.359 mean_real/fake: -0.000/0.001 var_real/fake: 0.008/0.008 
[578/1000][15/16] KL_real/fake: 4.363/4.359 mean_real/fake: -0.000/-0.001 var_real/fake: 0.00

[650/1000][15/16] KL_real/fake: 4.364/4.359 mean_real/fake: 0.000/0.002 var_real/fake: 0.008/0.008 
[651/1000][15/16] KL_real/fake: 4.362/4.358 mean_real/fake: -0.001/-0.000 var_real/fake: 0.008/0.008 
[652/1000][15/16] KL_real/fake: 4.363/4.359 mean_real/fake: -0.001/0.000 var_real/fake: 0.008/0.008 
[653/1000][15/16] KL_real/fake: 4.364/4.359 mean_real/fake: 0.000/-0.001 var_real/fake: 0.008/0.008 
[654/1000][15/16] KL_real/fake: 4.364/4.358 mean_real/fake: 0.000/0.001 var_real/fake: 0.008/0.008 
[655/1000][15/16] KL_real/fake: 4.362/4.358 mean_real/fake: 0.000/-0.001 var_real/fake: 0.008/0.008 
[656/1000][15/16] KL_real/fake: 4.362/4.358 mean_real/fake: -0.001/0.000 var_real/fake: 0.008/0.008 
[657/1000][15/16] KL_real/fake: 4.362/4.359 mean_real/fake: -0.000/-0.000 var_real/fake: 0.008/0.008 
[658/1000][15/16] KL_real/fake: 4.362/4.358 mean_real/fake: -0.001/0.001 var_real/fake: 0.008/0.008 
[659/1000][15/16] KL_real/fake: 4.361/4.357 mean_real/fake: -0.000/-0.000 var_real/fake: 0.

[731/1000][15/16] KL_real/fake: 4.361/4.357 mean_real/fake: 0.000/0.000 var_real/fake: 0.008/0.008 
[732/1000][15/16] KL_real/fake: 4.364/4.359 mean_real/fake: 0.001/0.000 var_real/fake: 0.008/0.008 
[733/1000][15/16] KL_real/fake: 4.361/4.358 mean_real/fake: -0.000/-0.002 var_real/fake: 0.008/0.008 
[734/1000][15/16] KL_real/fake: 4.361/4.357 mean_real/fake: -0.000/0.001 var_real/fake: 0.008/0.008 
[735/1000][15/16] KL_real/fake: 4.361/4.358 mean_real/fake: 0.001/0.001 var_real/fake: 0.008/0.008 
[736/1000][15/16] KL_real/fake: 4.361/4.359 mean_real/fake: -0.001/0.001 var_real/fake: 0.008/0.008 
[737/1000][15/16] KL_real/fake: 4.362/4.358 mean_real/fake: 0.000/0.000 var_real/fake: 0.008/0.008 
[738/1000][15/16] KL_real/fake: 4.362/4.359 mean_real/fake: -0.001/-0.000 var_real/fake: 0.008/0.008 
[739/1000][15/16] KL_real/fake: 4.361/4.358 mean_real/fake: -0.000/-0.000 var_real/fake: 0.008/0.008 
[740/1000][15/16] KL_real/fake: 4.363/4.358 mean_real/fake: -0.001/-0.001 var_real/fake: 0.0

[812/1000][15/16] KL_real/fake: 4.364/4.359 mean_real/fake: -0.001/-0.001 var_real/fake: 0.008/0.008 
[813/1000][15/16] KL_real/fake: 4.363/4.358 mean_real/fake: 0.001/0.001 var_real/fake: 0.008/0.008 
[814/1000][15/16] KL_real/fake: 4.362/4.359 mean_real/fake: 0.000/-0.001 var_real/fake: 0.008/0.008 
[815/1000][15/16] KL_real/fake: 4.361/4.359 mean_real/fake: 0.000/-0.001 var_real/fake: 0.008/0.008 
[816/1000][15/16] KL_real/fake: 4.365/4.358 mean_real/fake: -0.000/0.001 var_real/fake: 0.008/0.008 
[817/1000][15/16] KL_real/fake: 4.362/4.359 mean_real/fake: -0.000/-0.001 var_real/fake: 0.008/0.008 
[818/1000][15/16] KL_real/fake: 4.364/4.358 mean_real/fake: -0.000/0.001 var_real/fake: 0.008/0.008 
[819/1000][15/16] KL_real/fake: 4.361/4.359 mean_real/fake: 0.000/0.002 var_real/fake: 0.008/0.008 
[820/1000][15/16] KL_real/fake: 4.363/4.358 mean_real/fake: -0.000/-0.001 var_real/fake: 0.008/0.008 
[821/1000][15/16] KL_real/fake: 4.365/4.358 mean_real/fake: -0.000/-0.001 var_real/fake: 0

[893/1000][15/16] KL_real/fake: 4.363/4.359 mean_real/fake: -0.000/0.000 var_real/fake: 0.008/0.008 
[894/1000][15/16] KL_real/fake: 4.361/4.358 mean_real/fake: -0.000/0.000 var_real/fake: 0.008/0.008 
[895/1000][15/16] KL_real/fake: 4.361/4.359 mean_real/fake: -0.000/0.002 var_real/fake: 0.008/0.008 
[896/1000][15/16] KL_real/fake: 4.363/4.358 mean_real/fake: 0.001/0.001 var_real/fake: 0.008/0.008 
[897/1000][15/16] KL_real/fake: 4.363/4.358 mean_real/fake: -0.000/0.000 var_real/fake: 0.008/0.008 
[898/1000][15/16] KL_real/fake: 4.362/4.359 mean_real/fake: -0.000/-0.001 var_real/fake: 0.008/0.008 
[899/1000][15/16] KL_real/fake: 4.363/4.359 mean_real/fake: -0.000/0.002 var_real/fake: 0.008/0.008 
[900/1000][15/16] KL_real/fake: 4.362/4.357 mean_real/fake: 0.001/-0.000 var_real/fake: 0.008/0.008 
[901/1000][15/16] KL_real/fake: 4.365/4.358 mean_real/fake: 0.001/0.002 var_real/fake: 0.008/0.008 
[902/1000][15/16] KL_real/fake: 4.362/4.357 mean_real/fake: -0.000/-0.000 var_real/fake: 0.0

In [ ]:
# e_updates = '1;KL_fake:1,KL_real:1,match_z:0,match_x:10' 
# g_updates = '3;KL_fake:1,match_z:1000,match_x:0'

# 10 * 50 * 64 - 32000 
# 10 * 25 * 128
# 10 * 13 * 256

# https://github.com/pytorch/pytorch/issues/1355


In [ ]:
def train_hist(hist, run, show=True, save=True, o_p='output/' + dataset):
    
    plt.figure(figsize = (12, 6))
    x = range(len(hist['KL_real']))
    for i, j in hist.items():
        if j: 
            plt.plot(x, j, label=i)
            
#     [plt.plot(x, j, label=i) for i, j in hist.items() if j]

    plt.xlabel('Epoch')
    plt.ylabel('Losses')
    plt.legend(loc=4)
    plt.grid(True)
    plt.tight_layout()

    if save: plt.savefig(os.path.join(o_p, 'train_hist_%s.png' %run))
    if show: plt.show()
    else: plt.close()
    
def log_last_epoch_ims(run, o_p='output/' + dataset):
    ne = nepoch - 1
    if ne < 10: f_name = os.path.join(o_p, 'reconstructions_epoch_00' + str(ne) + '.png')
    elif ne < 100: f_name = os.path.join(o_p, 'reconstructions_epoch_0' + str(ne) + '.png')
    else: f_name = os.path.join(o_p, 'reconstructions_epoch_' + str(ne) + '.png')
    
    im = cv2.imread(f_name)
    cv2.imwrite(os.path.join(o_p, 'ims_last_epoch_%s.png' %run), im)
    plot_im(im, alpha=False)
    
train_hist(losses, run, save=True)
log_last_epoch_ims(run)

In [ ]:
def train_hist_ims_gif(otype, run, step=save_every_e, save=True, show=True, o_p = 'output/' + dataset):
    ims_rec = []
    i = 0
    while i < nepoch:
        if i < 10:
            f_name = os.path.join(o_p, otype + '_epoch_00' + str(i) + '.png')
            ims_rec.append(imageio.imread(f_name))
        elif i < 100:
            f_name = os.path.join(o_p, otype + '_epoch_0' + str(i) + '.png')
            ims_rec.append(imageio.imread(f_name))                     
        else:
            f_name = os.path.join(o_p, otype + '_epoch_' + str(i) + '.png')
            ims_rec.append(imageio.imread(f_name))  
            
        i += save_every_e
#         print(len(ims_rec))
        
    o_f = os.path.join(o_p, otype + '_train_hist_ims_%s.gif' %run)

    if save: 
        imageio.mimsave(o_f, ims_rec, fps=5)
    
    if show:
        from IPython.display import HTML
        HTML('<img src="%s">' %o_f)

for j in ['reconstructions', 'fake_samples']:
    train_hist_ims_gif(j, run)